In [2]:
'''
Analysis of /r/nba Posts
@author: Brian Lin
'''
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

In [3]:
def return_csv_files(path):
    pattern = r'^.*\.csv$'
    return [f for f in os.listdir(path) if bool(re.match(pattern,f))]
    
path = 'data/backlog/'
csv_files = return_csv_files(path)
df = pd.DataFrame()
for csv in csv_files:
    csv_df = pd.read_csv(path + csv)
    df = pd.concat([df,csv_df])
df['created_timestamp'] = pd.to_datetime(df.created, unit = 's')
df.index = range(0,len(df))
# 18,001 total examples with no duplicates

In [4]:
''' 
Randomly shuffle the dataframe and generates a training, cross-validation, and test set
'''
random_df = df.reindex(np.random.permutation(df.index))
training_df = random_df.iloc[0:12000]
cross_validation_df = random_df.iloc[12000:15000]
test_df = random_df.iloc[15000:]

In [5]:
def front_page(score,n=150):
    if score >= n:
        return 1
    else:
        return 0
training_Y = training_df.score.apply(front_page)
cross_validation_Y = cross_validation_df.score.apply(front_page)
test_Y = test_df.score.apply(front_page)

In [6]:
common_words = pd.read_csv('common_words.txt', header = None)[0].values

def remove_common_words(title, common_words):
    tw = map(lambda d: d.strip(".,:()[]!?\"").lower(),title.split())
    return [w for w in tw if w not in common_words]

def generate_words_and_scores(titles, score, common_words):
    title_words = []
    score_words = []
    for index in range(len(titles)):
        title_word_list = remove_common_words(titles.iloc[index], common_words)
        title_words.extend(title_word_list)
        score_words.extend([scores.iloc[index]] * len(title_word_list))
    title_words_s = pd.Series(title_words)
    words_scores_df = pd.concat([title_words_s, pd.Series(score_words)], axis = 1)
    words_scores_df.columns = ['word', 'score']
    return words_scores_df

titles = training_df.title
scores = training_df.score
word_df = generate_words_and_scores(titles,scores, common_words)
word_df_agg = word_df.groupby('word').agg({'score':['mean','std','median','count']})

In [7]:
words_sorted_median = word_df_agg[word_df_agg[('score','count')] > 10].sort_values(by=[('score','median')], ascending=False)
top200 = np.array(words_sorted_median[:200].index)
bottom200 = np.array(words_sorted_median[-200:].index)
top_bottom200 = np.array(pd.concat([words_sorted_median[:200],words_sorted_median[-200:]]).index)

In [9]:
#Generates the feature word vectors
def create_feature_row(title, word_vec):
    feature_row = np.zeros((len(word_vec),))
    for word in title:
        if word in word_vec:
            word_index = np.where(word_vec==word)[0][0]
            feature_row[word_index] = 1
    return feature_row

def generate_feature_vector(title_s, top_words, common_words):
    l = []
    for title in title_s:
        title_r = remove_common_words(title, common_words)
        l.append(create_feature_row(title_r, top_words))
    return l

def sanitize_titles(title_s):
    l = []
    for title in title_s:
        title_r = remove_common_words(title, common_words)
        l.append(title_r)
    return l
    


In [10]:
def prediction_score(predicted_p, Y,threshold=0.5):
    def greater_than_one_half(x):
        if x > threshold:
            return 1
        else:
            return 0
        
    predicted_Y = np.array([greater_than_one_half(x) for x in predicted_p])
    true_pos = 0
    false_pos = 0
    true_neg = 0
    false_neg = 0
    for i in range(len(predicted_Y)):
        if predicted_Y[i] == 1 and Y[i] == 1:
            true_pos +=1
        elif predicted_Y[i] == 1 and Y[i] == 0:
            false_pos +=1
        elif predicted_Y[i] == 0 and Y[i] == 1:
            true_neg +=1
        elif predicted_Y[i] == 0 and Y[i] == 0:
            false_neg +=1
    
    wrong = float(np.sum(np.abs(predicted_Y - Y)))
    total = float(len(predicted_Y))
    ans = dict()
    ans['accuracy'] =  1-(wrong/total)
    ans['true_pos'] = true_pos
    ans['true_neg'] = true_neg
    ans['false_pos'] = false_pos
    ans['false_neg'] = false_neg
    if true_pos == 0 and false_pos == 0:
        ans['recall'] = float(true_pos) / float((true_pos + true_neg))
    elif true_pos == 0 and true_neg == 0:
        ans['precision'] = float(true_pos) / float((true_pos + false_pos))
    else:
        ans['precision'] = float(true_pos) / float((true_pos + false_pos))
        ans['recall'] = float(true_pos) / float((true_pos + true_neg))
        ans['f1_score'] = 2 * ans['precision'] * ans['recall'] / (ans['precision'] + ans['recall'] )
    return ans

In [11]:
def generate_bayes_prob(words, X, Y):
    pxy1 = {}
    pxy0 = {}
    for index in range(len(words)):
        word = words[index]
        pxy1[word] = float(sum((X.iloc[:,index] == 1) & (Y == 1)) +1) / float(sum(Y.values == 1) +2)
        pxy0[word] = float(sum((X.iloc[:,index] == 1) & (Y == 0)) +1) / float(sum(Y.values == 0) +2)
    return pxy1, pxy0

def calculate_prob(sanitized_title_list,debug=False):
    global bayes_pxy1,bayes_pxy0, py1, py0,pno1, pno0
    pyx1 = 1.0
    pyx0 = 1.0
    for word in sanitized_title_list:
        if word in bayes_pxy1:
            pyx1 *= bayes_pxy1[word]
            pyx0 *= bayes_pxy0[word]
            if debug:
                print 'yes'
        else:
            pyx1 *= pno1
            pyx0 *= pno0
            if debug:
                print 'no'    
        if debug:
            print word, pyx1, pyx0
    return pyx1*py1, pyx0*py0

In [12]:
word_vector = np.array(word_df_agg.index)
#word_vector = top_bottom200
title_features_bayes_df = pd.DataFrame(generate_feature_vector(titles,word_vector,common_words), index = training_df.index)
sum(title_features_bayes_df.sum(axis =1) == 0)
bayes_pxy1,bayes_pxy0 = generate_bayes_prob(word_vector, title_features_bayes_df, training_Y)
Y = training_Y
py1 = float(sum(Y.values == 1)) / len(Y)
py0 = float(sum(Y.values == 0)) / len(Y)
pno1 = 1/float(sum(Y.values == 1))
pno0 = 1/float(sum(Y.values == 0))

In [13]:
sanitized_titles = sanitize_titles(training_df.title)
class_probs = []
for i in range(len(sanitized_titles)):
    title = sanitized_titles[i]
    class_probs.append(calculate_prob(title))
probs = pd.DataFrame(class_probs)
sum(probs[0] > probs[1])

7124

In [14]:
probs.index = training_df.index
pd.concat([training_df[probs[0] > probs[1]][['title','score']], probs], axis=1, join= 'inner').head()

,title,score,0,1
14939,Faking fivethrityeight.com's CARMELO -- 2016-1...,26.0,1.459184e-22,8.986296e-23
15268,Huertas comes up with the sneakiest steal I've...,23.0,7.264399e-23,2.395620e-23
15990,"Without contract, Smith to skip Cavs minicamp",238.0,8.490667e-13,1.745827e-13
3027,[Jon Wilner] Warriors GM Myers says no offseas...,20.0,6.526078e-24,5.287127e-24
1111,Lebron James Gets in Stephen Curry's Face - Ga...,168.0,1.112694e-15,6.451405e-17


In [15]:
import datetime
#2016-06-08 06:26:36
training_df[(Y.values == 0)].created.apply(lambda d: d.hour).plot(kind='hist')

AttributeError: 'float' object has no attribute 'hour'

In [ ]:
training_df[(predicted_Y3 == 1) & (Y.values == 1)][['title','score','domain','created']].domain.value_counts()[:15].plot(kind='barh')

In [ ]:
Y_predicted3 = probs.apply(lambda d: 1 if d[0] > d[1] else 0, axis = 1)
predicted_Y3 = np.array(Y_predicted3)
p_score = prediction_score(predicted_Y3, Y.values)
p_score